We solve the Neoclassical Growth Model
\begin{align}
  \rho V(k) &= \max_{c} u(c) +  V'(k)(F(k)-\delta k - c) \\
  &u(c) = \frac{c^{1-\sigma}}{1-\sigma},~~~~ F(k)=k^\alpha
\end{align}

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
sigma = 2 # 1 changes to log utility
alpha = 0.3
delta = 0.05
rho = 0.05

In [3]:
kss = (alpha/(rho+delta))**(1/(1-alpha))
kmin = 0.001*kss
kmax = 2*kss

In [4]:
train_num = 50000
train = np.random.uniform(kmin,kmax,(train_num,1))
# train = np.linspace(kmin, kmax, num=train_num)

In [5]:
def build_model():
  model = keras.Sequential([
    layers.Dense(2, activation='relu', input_shape=[1]), #use_bias=True
    layers.Dense(4, activation='relu'), #use_bias=False     
    layers.Dense(8, activation='relu'), #use_bias=False
    layers.Dense(16, activation='relu'), #use_bias=False     
    layers.Dense(8, activation='relu'), #use_bias=False 
    layers.Dense(4, activation='relu'), #use_bias=False 
    layers.Dense(2, activation='relu'), #use_bias=False   
    layers.Dense(1) #,kernel_constraint=keras.constraints.NonNeg()
  ])

  optimizer = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
#   optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                )
  return model

In [6]:
model = build_model()

In [7]:
model.summary()
# model.get_weights() 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 2)                 4         
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 12        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 40        
_________________________________________________________________
dense_3 (Dense)              (None, 16)                144       
_________________________________________________________________
dense_4 (Dense)              (None, 8)                 136       
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 36        
_________________________________________________________________
dense_6 (Dense)              (None, 2)                 1

In [8]:
initial_guess = -24+ train
model.fit( train, initial_guess, epochs=200, batch_size=1000,  validation_split = 0.2, verbose=1 )

Train on 40000 samples, validate on 10000 samples
Epoch 1/200
40000/40000 [==============================] - 1s 13us/sample - loss: 373.9791 - val_loss: 369.4063
Epoch 2/200
40000/40000 [==============================] - 0s 3us/sample - loss: 362.1366 - val_loss: 349.2700
Epoch 3/200
40000/40000 [==============================] - 0s 3us/sample - loss: 318.6531 - val_loss: 267.9897
Epoch 4/200
40000/40000 [==============================] - 0s 3us/sample - loss: 184.5324 - val_loss: 96.8009
Epoch 5/200
40000/40000 [==============================] - 0s 3us/sample - loss: 78.9855 - val_loss: 71.7333
Epoch 6/200
40000/40000 [==============================] - 0s 3us/sample - loss: 67.8249 - val_loss: 62.5708
Epoch 7/200
40000/40000 [==============================] - 0s 3us/sample - loss: 58.7439 - val_loss: 53.7314
Epoch 8/200
40000/40000 [==============================] - 0s 3us/sample - loss: 49.7815 - val_loss: 44.8225
Epoch 9/200
40000/40000 [==============================] - 0s 3us/samp

40000/40000 [==============================] - 0s 3us/sample - loss: 4.1095e-07 - val_loss: 3.9953e-07
Epoch 74/200
40000/40000 [==============================] - 0s 3us/sample - loss: 3.7039e-07 - val_loss: 3.5865e-07
Epoch 75/200
40000/40000 [==============================] - 0s 4us/sample - loss: 3.3634e-07 - val_loss: 3.2152e-07
Epoch 76/200
40000/40000 [==============================] - 0s 4us/sample - loss: 3.0454e-07 - val_loss: 2.8970e-07
Epoch 77/200
40000/40000 [==============================] - 0s 3us/sample - loss: 2.7604e-07 - val_loss: 2.6294e-07
Epoch 78/200
40000/40000 [==============================] - 0s 3us/sample - loss: 2.5241e-07 - val_loss: 2.3529e-07
Epoch 79/200
40000/40000 [==============================] - 0s 3us/sample - loss: 2.2983e-07 - val_loss: 2.1199e-07
Epoch 80/200
40000/40000 [==============================] - 0s 3us/sample - loss: 2.0947e-07 - val_loss: 1.9261e-07
Epoch 81/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.9

40000/40000 [==============================] - 0s 3us/sample - loss: 2.2855e-09 - val_loss: 5.5861e-10
Epoch 144/200
40000/40000 [==============================] - 0s 3us/sample - loss: 2.0642e-09 - val_loss: 6.9802e-10
Epoch 145/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.9688e-09 - val_loss: 4.1487e-10
Epoch 146/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.8217e-09 - val_loss: 3.4336e-10
Epoch 147/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.6852e-09 - val_loss: 3.2714e-10
Epoch 148/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.6275e-09 - val_loss: 2.8937e-10
Epoch 149/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.7867e-09 - val_loss: 8.0864e-10
Epoch 150/200
40000/40000 [==============================] - 0s 3us/sample - loss: 1.5201e-09 - val_loss: 1.3461e-09
Epoch 151/200
40000/40000 [==============================] - 0s 2us/sample - l

In [9]:
x_test = np.linspace(kmin,kmax,num=1000)

y_test = -24+ x_test

error = y_test - model.predict([x_test])

np.mean(abs(error))

3.2042337782451313

In [10]:
inputs = tf.constant(train)
with tf.GradientTape() as t: #(persistent=True)
    t.watch(inputs)
    output = model(inputs)
derivative = t.gradient(output, inputs).numpy()
print(derivative)

[[0.99999374]
 [0.99999374]
 [0.99999374]
 ...
 [0.99999374]
 [0.99999374]
 [0.99999374]]


In [11]:
# np.ones(train_num).reshape(train_num,1).shape

In [11]:
update = 10
dt = 5*1e-2

In [ ]:
for update in range(update):
        value = model.predict([train])
        print(type(value))

        with tf.GradientTape() as t: #(persistent=True)
            t.watch(inputs)
            output = model(inputs)
            
        derivative = t.gradient(output, inputs).numpy()
        print(derivative)

        for i in range(train_num):
            derivative[i] = np.maximum(derivative[i], 1e-4)
            
        # with log utility function
#         HJB = np.log(1/derivative)+derivative*((train)**alpha-delta*train-1/derivative)-rho*value

        # with sigma=2 CRRA utility function
        HJB = (1/np.sqrt(derivative))**(1-sigma)/(1-sigma)+derivative*((train)**alpha-delta*train-1/np.sqrt(derivative))-rho*value
#         print(HJB)
        value += dt*HJB
        model.fit( train, value,  epochs=2000, batch_size=2000, validation_split = 0.2, verbose=0 )
#         print(update,  max(abs(HJB)), np.mean(abs(HJB)))

<class 'numpy.ndarray'>
[[0.99999374]
 [0.99999374]
 [0.99999374]
 ...
 [0.99999374]
 [0.99999374]
 [0.99999374]]
<class 'numpy.ndarray'>
[[1.0011853]
 [1.0011853]
 [1.0011853]
 ...
 [1.0011853]
 [1.0011853]
 [1.0011853]]
<class 'numpy.ndarray'>
[[1.00577521]
 [0.99987453]
 [1.00577521]
 ...
 [0.99987453]
 [0.99987453]
 [0.99987453]]


In [14]:
x = np.linspace(kmin, kmax, num=100)
y = model.predict([x])
import matplotlib.pyplot as plt
plt.plot(x,y)

plt.xlabel('k')
plt.ylabel('V(k)')
plt.title('Value function')
plt.show()

<Figure size 640x480 with 1 Axes>